In [1]:
import scipy.io as spio
import scipy.signal as spsignal
import numpy as np
from ipynb.fs.full.cutoutTrials import *

def load_and_preprocessing_data():
    
    dataset_name = ['a', 'b', 'f', 'g']
    dataSet = spio.loadmat(f'BCICIV_calib_ds1{dataset_name[0]}.mat')
    butter = spio.loadmat('butter830.mat')

    butter_a = butter['a'][0]
    butter_b = butter['b'][0]

    dataSet['cnt'] = spsignal.lfilter(butter_a, butter_b, dataSet['cnt'])
    
    #------- Select channels and convert cnt into float ------

    vector = np.vectorize(np.float)
    x = dataSet['cnt']
    x = vector(x)
    dataSet['cnt'] = 0.1*x
    
    #-------- Cut EEG into tirals --------

    xcnt = dataSet['cnt']
    pos = dataSet['mrk']['pos'][0][0]
    opt_ival = [2000,6000]
    fs =  dataSet['nfo']['fs'][0][0][0][0]

    xepo = cutoutTrials(xcnt, pos, opt_ival, fs)
    
    #----- Find indices of training and test set------

    label = dataSet['mrk']['y'][0][0][0]
    xdata = xepo[:,:,:]
    label = (label/2) + 1.5
    
    return xdata, label
